In [ ]:
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

(train_input, train_target), (test_input, test_target) = imdb.load_data(num_words=500)

In [ ]:
train_input, val_input, train_target, val_target = train_test_split (
    train_input, train_target, test_size=0.2, random_state=42
)

In [ ]:
# 토큰의 개수를 맞추기 위해 maxlen으로 패딩을 하거나 자름 
# 추가하거나 제거하는 대상은 앞부분 
train_seq = pad_sequences(train_input, maxlen=100)
val_seq = pad_sequences(val_input, maxlen=100)
train_oh = keras.utils.to_categorical(train_seq)
val_oh = keras.utils.to_categorical(val_seq)

In [ ]:
# 원-핫 인코딩 모델
# SimpleRNN: 순환 신경망 층
# input_shape = (100,500) -> 100: length, 500: one-hot 인코딩으로 표현하기 위한 배열의 길이 
# 토큰을 변환한 정수는 특별한 의미가 없으므로 one-hot 인코딩을 사용해야 함 (e.g., 20이 10보다 중요한 것이 아님, 즉 크기 속성을 없애는 것)
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(8, input_shape=(100,500)))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
# 학습률 0.0001의 RMSprop 옵티마이저 사용
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-simplernn-model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

history = model.fit(train_oh, train_target, epochs=100, batch_size=64,
                    validation_data = (val_oh, val_target),
                    callbacks = [checkpoint_cb, early_stopping_cb])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','val'])
plt.show()

In [ ]:
# 단어 임베딩 모델 
# (어휘 사전의 크기, 임베딩 벡터의 크기, 샘플의 길이)
model2 = keras.Sequential()
model2.add(keras.layers.Embedding(500, 16, input_length=100))
model2.add(keras.layers.SimpleRNN(8))
model2.add(keras.layers.Dense(1, activation='sigmoid'))
model2.summary()

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model2.compile(optimizer=rmsprop, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-embedding-model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
history = model2.fit(train_seq, train_target, epochs=100, batch_size=64,
                     validation_data = (val_seq, val_target),
                     callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','val'])
plt.show()